<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Business Analytics e Machine Learning Para Projetos de Data Science</font>
## <font color='blue'>Projeto 2</font>
### <font color='blue'>Template de Projetos de Data Science - Análise de Sentimento em Avaliações de Usuários</font>

## Etapa 1 - Definição de Objetivos de Negócio e Requisitos de Dados

Tudo começar por aqui. Veja a descrição desta etapa no ebook no Capítulo 8.

## Etapa 2 - Mapeamento do Fluxo de Dados e Processos de Negócio

Veja a descrição desta etapa no ebook no Capítulo 8.

## Etapa 3 - Análise Exploratória de Dados (EDA) com Python

## Instalando e Carregando os Pacotes

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [2]:
# Imports
import re
import pickle
import nltk
import sklearn
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [4]:
# Carrega o dataset
df_dsa = pd.read_csv('dataset.csv')

In [5]:
# Shape
df_dsa.shape

(50000, 2)

In [6]:
# Amostra dos dados
df_dsa.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
# Info
df_dsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [8]:
# Contagem de registros por classe
df_dsa.sentiment.value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

## Etapa 4 - Limpeza dos Dados

In [9]:
# Ajustamos os labels para representação numérica
df_dsa.sentiment.replace('positive', 1, inplace = True)
df_dsa.sentiment.replace('negative', 0, inplace = True)

In [10]:
# Amostra dos dados
df_dsa.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [11]:
# Vamos observar uma avaliação de usuário
df_dsa.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [12]:
# Função de limpeza geral de dados
def dsa_limpa_dados(texto):
    cleaned = re.compile(r'<.*?>') 
    return re.sub(cleaned, '', texto)

A função acima recebe um argumento chamado texto, que é a string de texto a ser limpa. Dentro da função, a expressão regular r'<.*?>' é compilada. Esta expressão regular corresponde a qualquer coisa que esteja entre os caracteres < e >, incluindo os próprios caracteres de abertura e fechamento de tags. O ponto (.) corresponde a qualquer caractere (exceto quebras de linha), e o asterisco (*) indica que o caractere anterior (neste caso, o ponto) pode ocorrer zero ou mais vezes. O ponto de interrogação (?) torna a correspondência "preguiçosa", o que significa que ela corresponderá ao menor número possível de caracteres.

A função re.sub() é usada para substituir todas as ocorrências da expressão regular compilada (ou seja, as tags HTML) por uma string vazia (''). Isso efetivamente remove todas as tags HTML do texto. O texto limpo, sem as tags HTML, é retornado pela função.

In [13]:
# Testando a função
texto_com_tags = "<p>Este é um exemplo <b>com</b> tags HTML.</p>"
texto_limpo = dsa_limpa_dados(texto_com_tags)
print(texto_limpo)  

Este é um exemplo com tags HTML.


In [14]:
# Aplica a função ao nosso dataset
df_dsa.review = df_dsa.review.apply(dsa_limpa_dados)

In [15]:
df_dsa.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [16]:
# Função para limpeza de caracteres especiais
def dsa_limpa_caracter_especial(texto):
    rem = ''
    for i in texto:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
            
    return rem

Na função acima, a variável rem é inicializada como uma string vazia (''). Ela será usada para armazenar o texto resultante após a remoção dos caracteres especiais. A condição if i.isalnum() verifica se o caractere i é alfanumérico (ou seja, se é uma letra ou um número). A função isalnum() retorna True se o caractere for alfanumérico e False caso contrário. Se o caractere i for alfanumérico, ele é adicionado à variável rem. Se não for alfanumérico, um espaço (' ') é adicionado a rem no lugar do caractere especial. Após iterar por todos os caracteres da string texto, a função retorna a variável rem, que contém o texto com os caracteres especiais substituídos por espaços.

In [17]:
# Testando a função
texto_com_caracteres_especiais = "Olá, mundo! Como vai?"
texto_limpo = dsa_limpa_caracter_especial(texto_com_caracteres_especiais)
print(texto_limpo)

Olá  mundo  Como vai 


In [18]:
# Aplica a função
df_dsa.review = df_dsa.review.apply(dsa_limpa_caracter_especial)

In [19]:
df_dsa.review[0]

'One of the other reviewers has mentioned that after watching just 1 Oz episode you ll be hooked  They are right  as this is exactly what happened with me The first thing that struck me about Oz was its brutality and unflinching scenes of violence  which set in right from the word GO  Trust me  this is not a show for the faint hearted or timid  This show pulls no punches with regards to drugs  sex or violence  Its is hardcore  in the classic use of the word It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary  It focuses mainly on Emerald City  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  Em City is home to many  Aryans  Muslims  gangstas  Latinos  Christians  Italians  Irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [20]:
# Função para converter o texto em minúsculo
def dsa_converte_minusculo(texto):
    return texto.lower()

Converter todo o texto para minúsculo ajuda a padronizar os dados, tornando-os mais consistentes. Isso é especialmente útil quando se lida com texto proveniente de diferentes fontes ou formatos.

In [21]:
# Testando a função
frase = "Esta é uma fraSE com LETRAS MaiúscuLAs"
frase_saida = dsa_converte_minusculo(frase)
print(frase_saida)

esta é uma frase com letras maiúsculas


In [22]:
# Aplica a função
df_dsa.review = df_dsa.review .apply(dsa_converte_minusculo)

In [23]:
df_dsa.review[0]

'one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked  they are right  as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [24]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/dmpm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/dmpm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
# Função para remover stopwords
def dsa_remove_stopwords(texto):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(str(texto))
    return [w for w in words if w not in stop_words]

A função acima utiliza a biblioteca NLTK (Natural Language Toolkit) para obter um conjunto de stopwords para o idioma inglês com stopwords.words('english'). Essas stopwords incluem palavras comuns como "the", "is", "in", etc., que geralmente não contribuem para o significado principal do texto em análises de PLN.

O texto é convertido em uma lista de palavras (ou "tokens") usando a função word_tokenize da NLTK. A função word_tokenize divide o texto em palavras com base em espaços e pontuação.

A list comprehension [w for w in words if w not in stop_words] é usada para criar uma nova lista de palavras que contém apenas aquelas palavras que não estão presentes no conjunto de stopwords. Ou seja, todas as palavras que são consideradas irrelevantes são removidas do texto.

In [26]:
# Testando a função
frase =  "They are right, as this is exactly what happened with me."
frase_saida = dsa_remove_stopwords(frase)
print(frase_saida)

['They', 'right', ',', 'exactly', 'happened', '.']


In [27]:
%%time
df_dsa.review = df_dsa.review.apply(dsa_remove_stopwords)

CPU times: user 14.5 s, sys: 439 ms, total: 14.9 s
Wall time: 15 s


In [28]:
df_dsa.review[0]

['one',
 'reviewers',
 'mentioned',
 'watching',
 '1',
 'oz',
 'episode',
 'hooked',
 'right',
 'exactly',
 'happened',
 'first',
 'thing',
 'struck',
 'oz',
 'brutality',
 'unflinching',
 'scenes',
 'violence',
 'set',
 'right',
 'word',
 'go',
 'trust',
 'show',
 'faint',
 'hearted',
 'timid',
 'show',
 'pulls',
 'punches',
 'regards',
 'drugs',
 'sex',
 'violence',
 'hardcore',
 'classic',
 'use',
 'word',
 'called',
 'oz',
 'nickname',
 'given',
 'oswald',
 'maximum',
 'security',
 'state',
 'penitentary',
 'focuses',
 'mainly',
 'emerald',
 'city',
 'experimental',
 'section',
 'prison',
 'cells',
 'glass',
 'fronts',
 'face',
 'inwards',
 'privacy',
 'high',
 'agenda',
 'em',
 'city',
 'home',
 'many',
 'aryans',
 'muslims',
 'gangstas',
 'latinos',
 'christians',
 'italians',
 'irish',
 'scuffles',
 'death',
 'stares',
 'dodgy',
 'dealings',
 'shady',
 'agreements',
 'never',
 'far',
 'away',
 'would',
 'say',
 'main',
 'appeal',
 'show',
 'due',
 'fact',
 'goes',
 'shows',
 'da

In [29]:
# Função para o stemmer
def dsa_stemmer(texto):
    objeto_stemmer = SnowballStemmer('english')
    return " ".join([objeto_stemmer.stem(w) for w in texto])

O "stemming" é uma técnica de Processamento de Linguagem Natural (PLN) usada para reduzir as palavras à sua forma raiz ou base, removendo prefixos e sufixos comuns. Isso ajuda a normalizar as palavras para análise e pode melhorar o desempenho de tarefas de PLN, como pesquisa de texto, classificação de texto e agrupamento de documentos.

A função acima recebe um argumento, que é a string de texto a ser processada. Um objeto SnowballStemmer é criado para o idioma inglês ('english'). Esse objeto será usado para realizar o stemming das palavras.

A função usa um list comprehension para aplicar o método stem do objeto SnowballStemmer a cada palavra no texto. O método stem reduz cada palavra à sua forma base. O texto é dividido em palavras assumindo que ele já está tokenizado (separado em palavras individuais). Se o texto não estiver tokenizado, você precisará fazer isso antes de aplicar o stemmer.

In [30]:
# Testando a função
texto = "The cats are running"
texto_stemmed = dsa_stemmer(texto.split())
print(texto_stemmed)  

the cat are run


In [31]:
%%time
df_dsa.review = df_dsa.review.apply(dsa_stemmer)

CPU times: user 22.2 s, sys: 36.6 ms, total: 22.2 s
Wall time: 22.3 s


In [32]:
df_dsa.review[0]

'one review mention watch 1 oz episod hook right exact happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focus main emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch darker side'

## Etapa 5 - Pré-Processamento dos Dados

In [33]:
# Aumentar o valor de max_colwidth para evitar truncagem
pd.set_option('display.max_colwidth', 120)  

In [34]:
# Carrega o dataset original
dados_brutos = pd.read_csv('dataset.csv')

In [35]:
# Amostra dos dados brutos
dados_brutos.head(10)

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as ...",positive
1,A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and g...,positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater...",positive
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fightin...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portr...",positive
5,"Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's no...",positive
6,I sure would like to see a resurrection of a up dated Seahunt series with the tech they have today it would bring ba...,positive
7,"This show was an amazing, fresh & innovative idea in the 70's when it first aired. The first 7 or 8 years were brill...",negative
8,Encouraged by the positive comments about this film on here I was looking forward to watching this film. Bad mistake...,negative
9,If you like original gut wrenching laughter you will like this movie. If you are young or old then you will love thi...,positive


In [36]:
# Amostra dos dados limpos
df_dsa.head(10)

,review,sentiment
0,one review mention watch 1 oz episod hook right exact happen first thing struck oz brutal unflinch scene violenc set...,1
1,wonder littl product film techniqu unassum old time bbc fashion give comfort sometim discomfort sens realism entir p...,1
2,thought wonder way spend time hot summer weekend sit air condit theater watch light heart comedi plot simplist dialo...,1
3,basic famili littl boy jake think zombi closet parent fight time movi slower soap opera sudden jake decid becom ramb...,0
4,petter mattei love time money visual stun film watch mr mattei offer us vivid portrait human relat movi seem tell us...,1
5,probabl time favorit movi stori selfless sacrific dedic nobl caus preachi bore never get old despit seen 15 time las...,1
6,sure would like see resurrect date seahunt seri tech today would bring back kid excit grew black white tv seahunt gu...,1
7,show amaz fresh innov idea 70 first air first 7 8 year brilliant thing drop 1990 show realli funni anymor continu de...,0
8,encourag posit comment film look forward watch film bad mistak seen 950 film truli one worst aw almost everi way edi...,0
9,like origin gut wrench laughter like movi young old love movi hell even mom like great camp,1


In [37]:
# Podemos deletar o dataframe para liberar memória
del dados_brutos

In [38]:
# Extrai o texto da avaliação (entrada)
x = np.array(df_dsa.iloc[:,0].values)

In [39]:
# Extrai o sentimento (saída)
y = np.array(df_dsa.sentiment.values)

In [40]:
# Divisão dos dados em treino e teste com proporção 80/20
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size = 0.2, random_state = 0)

In [41]:
type(x_treino)

numpy.ndarray

In [42]:
# Cria um vetorizador (vai converter os dados de texto em representação numérica)
vetorizador_dsa = CountVectorizer(max_features = 1000)

In [43]:
# Fit e transform do vetorizador com dados de treino
x_treino_final = vetorizador_dsa.fit_transform(x_treino).toarray()

In [44]:
# Apenas transorm nos dados de teste
x_teste_final = vetorizador_dsa.transform(x_teste).toarray()

In [45]:
print("x_treino_final:", x_treino_final.shape)
print("y_treino:", y_treino.shape)

x_treino_final: (40000, 1000)
y_treino: (40000,)


In [46]:
print(x_treino_final)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [47]:
print("x_teste_final:", x_teste_final.shape)
print("y_teste:", y_teste.shape)

x_teste_final: (10000, 1000)
y_teste: (10000,)


## Etapa 6 - Criação de Modelos de Machine Learning

### Modelo Probabilístico 1 - GaussianNB

O Gaussian Naive Bayes (GaussianNB) é um modelo probabilístico baseado no teorema de Bayes, com a suposição de que as variáveis preditoras seguem uma distribuição gaussiana (normal). É frequentemente usado em problemas de classificação onde as características são contínuas e se presume que seguem uma distribuição normal. Este modelo é particularmente útil para classificar dados com características que variam de maneira contínua e são facilmente ajustáveis à curva gaussiana, tornando-o adequado para muitos cenários de Ciência de Dados e Machine Learning, especialmente em tarefas de classificação.

In [48]:
# Cria o modelo
modelo_v1 = GaussianNB()

In [49]:
# Treina o modelo
modelo_v1.fit(x_treino_final, y_treino)

GaussianNB()

### Modelo Probabilístico 2 - MultinomialNB

Multinomial Naive Bayes (MultinomialNB) é outro modelo baseado no teorema de Bayes, mas é particularmente adequado para contar dados ou características que representam contagens ou frequências de eventos. Este modelo é comumente utilizado em tarefas de classificação de texto, onde as características podem ser, por exemplo, a frequência de palavras ou a contagem de termos em documentos. Ele lida bem com a característica de os dados serem representados em forma de vetores de contagens ou frequências, tornando-o altamente eficaz para a filtragem de spam, análise de sentimentos e categorização de documentos.

In [50]:
# Cria o modelo
modelo_v2 = MultinomialNB(alpha = 1.0, fit_prior = True)

In [51]:
# Treina o modelo
modelo_v2.fit(x_treino_final, y_treino)

MultinomialNB()

### Modelo Probabilístico 3 - BernoulliNB

O Bernoulli Naive Bayes (BernoulliNB) é um modelo probabilístico que também utiliza o teorema de Bayes, mas é otimizado para dados binários/booleanos. Este modelo assume que todas as características são independentes e seguem uma distribuição de Bernoulli, o que significa que cada característica é representada por uma variável aleatória binária que pode assumir apenas dois possíveis resultados (por exemplo, 0 ou 1, verdadeiro ou falso). É particularmente útil em situações onde as características são explicitamente binárias, como em tarefas de classificação de texto, onde se considera apenas a presença ou ausência de uma palavra no documento, ignorando sua frequência.

In [52]:
# Cria o modelo
modelo_v3 = BernoulliNB(alpha = 1.0, fit_prior = True)

In [53]:
# Treina o modelo
modelo_v3.fit(x_treino_final, y_treino)

BernoulliNB()

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

## Etapa 7 - Avaliação, Interpretação e Comparação dos Modelos

In [54]:
# Previsões com dados de teste
ypred_v1 = modelo_v1.predict(x_teste_final)

In [55]:
# Previsões com dados de teste
ypred_v2 = modelo_v2.predict(x_teste_final)

In [56]:
# Previsões com dados de teste
ypred_v3 = modelo_v3.predict(x_teste_final)

In [57]:
print("Acurácia do Modelo GaussianNB = ", accuracy_score(y_teste, ypred_v1) * 100)
print("Acurácia do Modelo MultinomialNB = ", accuracy_score(y_teste, ypred_v2) * 100)
print("Acurácia do Modelo BernoulliNB = ", accuracy_score(y_teste, ypred_v3) * 100)

Acurácia do Modelo GaussianNB =  79.06
Acurácia do Modelo MultinomialNB =  82.57
Acurácia do Modelo BernoulliNB =  83.02000000000001


A acurácia é uma métrica global ideal para comparar versões do modelo do mesmo algoritmo. Para modelos de algoritmos diferentes a métrica AUC (Area Under The Curve) é a ideal.

In [58]:
# Import
from sklearn.metrics import roc_auc_score

In [59]:
# AUC do GaussianNB
y_proba = modelo_v1.predict_proba(x_teste_final)[:, 1]
auc = roc_auc_score(y_teste, y_proba)
print("AUC do Modelo GaussianNB =", auc)

AUC do Modelo GaussianNB = 0.861081232980416


In [60]:
# AUC do MultinomialNB
y_proba = modelo_v2.predict_proba(x_teste_final)[:, 1]
auc = roc_auc_score(y_teste, y_proba)
print("AUC do Modelo MultinomialNB =", auc)

AUC do Modelo MultinomialNB = 0.8993217067636314


In [61]:
# AUC do BernoulliNB
y_proba = modelo_v3.predict_proba(x_teste_final)[:, 1]
auc = roc_auc_score(y_teste, y_proba)
print("AUC do Modelo BernoulliNB =", auc)

AUC do Modelo BernoulliNB = 0.9083430688103717


In [62]:
# Salva o melhor modelo em disco
with open('modelo_v3.pkl', 'wb') as arquivo:
    pickle.dump(modelo_v3, arquivo)

## Etapa 8 - Deploy e Uso do Modelo

In [63]:
# Carrega o modelo do disco
with open('modelo_v3.pkl', 'rb') as arquivo:
    modelo_final = pickle.load(arquivo)

In [64]:
# Texto de uma avaliação de usuário (esse texto apresenta sentimento positivo)
texto_aval = """This is probably the fastest-paced and most action-packed of the German Edgar Wallace ""krimi"" 
series, a cross between the Dr. Mabuse films of yore and 60's pop thrillers like Batman and the Man 
from UNCLE. It reintroduces the outrageous villain from an earlier film who dons a stylish monk's habit and 
breaks the necks of victims with the curl of a deadly whip. Set at a posh girls' school filled with lecherous 
middle-aged professors, and with the cops fondling their hot-to-trot secretaries at every opportunity, it 
certainly is a throwback to those wonderfully politically-incorrect times. There's a definite link to a later 
Wallace-based film, the excellent giallo ""Whatever Happened to Solange?"", which also concerns female students 
being corrupted by (and corrupting?) their elders. Quite appropriate to the monk theme, the master-mind villain 
uses booby-trapped bibles here to deal some of the death blows, and also maintains a reptile-replete dungeon 
to amuse his captive audiences. <br /><br />Alfred Vohrer was always the most playful and visually flamboyant 
of the series directors, and here the lurid colour cinematography is the real star of the show. The Monk appears 
in a raving scarlet cowl and robe, tastefully setting off the lustrous white whip, while appearing against 
purplish-night backgrounds. There's also a voyeur-friendly turquoise swimming pool which looks great both 
as a glowing milieu for the nubile students and as a shadowy backdrop for one of the murder scenes. 
The trademark ""kicker"" of hiding the ""Ende"" card somewhere in the set of the last scene is also quite 
memorable here. And there's a fine brassy and twangy score for retro-music fans.<br /><br />Fans of the series 
will definitely miss the flippant Eddie Arent character in these later films. Instead, the chief inspector 
Sir John takes on the role of buffoon, convinced that he has mastered criminal psychology after taking a few 
night courses. Unfortunately, Klaus Kinski had also gone on to bigger and better things. The krimis had 
lost some of their offbeat subversive charm by this point, and now worked on a much more blatant pop-culture 
level, which will make this one quite accessible to uninitiated viewers."""

In [65]:
# Fluxo de transformação dos dados
tarefa1 = dsa_limpa_dados(texto_aval)
tarefa2 = dsa_limpa_caracter_especial(tarefa1)
tarefa3 = dsa_converte_minusculo(tarefa2)
tarefa4 = dsa_remove_stopwords(tarefa3)
tarefa5 = dsa_stemmer(tarefa4)

In [66]:
print(tarefa5)

probabl fastest pace action pack german edgar wallac krimi seri cross dr mabus film yore 60 pop thriller like batman man uncl reintroduc outrag villain earlier film don stylish monk habit break neck victim curl dead whip set posh girl school fill lecher middl age professor cop fondl hot trot secretari everi opportun certain throwback wonder polit incorrect time definit link later wallac base film excel giallo whatev happen solang also concern femal student corrupt corrupt elder quit appropri monk theme master mind villain use boobi trap bibl deal death blow also maintain reptil replet dungeon amus captiv audienc alfr vohrer alway play visual flamboy seri director lurid colour cinematographi real star show monk appear rave scarlet cowl robe tast set lustrous white whip appear purplish night background also voyeur friend turquois swim pool look great glow milieu nubil student shadowi backdrop one murder scene trademark kicker hide end card somewher set last scene also quit memor fine bra

In [67]:
type(tarefa5)

str

In [68]:
# Convertendo a string para um array Numpy (pois foi assim que o modelo foi treinado)
tarefa5_array = np.array(tarefa5)

In [69]:
type(tarefa5_array)

numpy.ndarray

In [70]:
# Aplicamos o vetorizador com mais uma conversão para array NumPy a fim de ajustar o shape de 0-d para 1-d
aval_final = vetorizador_dsa.transform(np.array([tarefa5_array])).toarray()

In [71]:
type(aval_final)

numpy.ndarray

In [72]:
# Previsão com o modelo
previsao = modelo_final.predict(aval_final.reshape(1,1000))

In [73]:
print(previsao)

[1]


In [74]:
# Estrutura condicional para verificar o valor de previsao
if previsao == 1:
    print("O Texto Indica Sentimento Positivo!")
else:
    print("O Texto Indica Sentimento Negativo!")

O Texto Indica Sentimento Positivo!


## Etapa 9 - Comunicação de Insights com Storytelling de Dados

Veja a descrição desta etapa no ebook no Capítulo 8.

## Etapa 10 - Documentação e Entrega de Projetos de Ciência de Dados

Veja a descrição desta etapa no ebook no Capítulo 8.

In [75]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [76]:
#%watermark -v -m

In [77]:
#%watermark --iversions

# Fim